In [ ]:
from labeyrieClasses import target #, fftw_psd
import numpy as np
import matplotlib.pyplot as plt
import ctypes


In [ ]:
binary = target()
binary.fits.fileName = "/home/niels/Documents/FITS/KP330.fits"
binary.fits.read(numDimensions = 3)

In [ ]:
# Testing PSD wrapper

# Use FFTW to calculate the PSD of a single image
# Input image = 512x512 ndarray of np.double32
# Output image = 

# Import shared C library
fftw_psd_dll = ctypes.CDLL('/home/niels/Dropbox/Thesis/Python/dev/fftw_psd.so')

# Image sizing parameters
imgsize = 512
psd_n = imgsize*(int(imgsize/2)+1)
img_n = imgsize**2



In [ ]:
# Get input file
input_img = binary.fits.data[0]
#array = np.zeros((imgsize,imgsize))
#array[int(imgsize/2)+1,int(imgsize/2)] = 1
#array[int(imgsize/2)-1,int(imgsize/2)] = 1
#array[int(imgsize/2),int(imgsize/2)] = 1

In [ ]:
# Reshape Square array to be flat
input_img_flat = np.reshape(input_img.astype(np.float32),(imgsize**2,1))

# Create pointers for in/out
img_ptr = (input_img_flat).ctypes.data_as(ctypes.POINTER(ctypes.c_float))
out_ptr = (np.zeros(img_n,np.float32)).ctypes.data_as(ctypes.POINTER(ctypes.c_float))
# Array type to be passed to wrapped C function
# Set input argument to be flat array of doubles (# of input img pixels)
fftw_psd_dll.psd.argtypes = [ctypes.POINTER(ctypes.c_float)]
fftw_psd_dll.psd.restype = ctypes.POINTER(ctypes.c_float * psd_n)

# Calculate PSD, get a pointer returned
out_ptr = fftw_psd_dll.psd(img_ptr)

# Reshape array to image
psd_image = np.reshape(out_ptr.contents,(imgsize,int(imgsize/2+1)))

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(np.reshape(input_img_flat,(imgsize,imgsize)))
plt.subplot(1,2,2)
plt.imshow(np.log10(psd_image))
if (imgsize < 16):
    print(psd_image)
    print(psd_array)
    print(array)

In [ ]:
def fftw_psd(input_img):
    # Reshape Square array to be flat
    input_img_flat = np.reshape(input_img.astype(np.float32),(imgsize**2,1))

    # Create pointers for in/out
    img_ptr = (input_img_flat).ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = (np.zeros(img_n,np.float32)).ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    # Array type to be passed to wrapped C function
    # Set input argument to be flat array of doubles (# of input img pixels)
    fftw_psd_dll.psd.argtypes = [ctypes.POINTER(ctypes.c_float)]
    fftw_psd_dll.psd.restype = ctypes.POINTER(ctypes.c_float * psd_n)

    # Calculate PSD, get a pointer returned
    out_ptr = fftw_psd_dll.psd(img_ptr)

    # Reshape array to image
    psd_image = np.reshape(out_ptr.contents,(imgsize,int(imgsize/2+1)))# Get input file
input_img = binary.fits.data[0]
#array = np.zeros((imgsize,imgsize))
#array[int(imgsize/2)+1,int(imgsize/2)] = 1
#array[int(imgsize/2)-1,int(imgsize/2)] = 1
#array[int(imgsize/2),int(imgsize/2)] = 1
    
    # Return PSD Image
    return psd_image

In [ ]:
# Averaging ~8.6ms for each PSD calculation

psd_sum = np.zeros((imgsize,int(imgsize/2)+1))
image = binary.fits.data[0]
%timeit psd_sum = fftw_psd(image)
